# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

### Get data from `.csv` files and then preprocess data

In [2]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    df_eng.columns = ["Title", "Year", "Synopsis", "Review"]
    df_eng["Original Language"] = "en"
    df_fr.columns = ["Title", "Year", "Synopsis", "Review",]
    df_fr["Original Language"] = "fr"
    df_sp.columns = ["Title", "Year", "Synopsis", "Review",]
    df_sp["Original Language"] = "sp"
    df = pd.concat([df_eng, df_fr, df_sp], ignore_index=True)
    return df

df = preprocess_data()

In [3]:
df.sample(10)

Title         Year  \
8              Solo: A Star Wars Story         2018   
28  Torrente: El brazo tonto de la ley         1998   
12                              Amélie         2001   
1                     The Dark Knight          2008   
0            The Shawshank Redemption          1994   
16      La Tour Montparnasse Infernale         2001   
26                             Toc Toc         2017   
25                        Águila Roja   (2009-2016)   
6          Scott Pilgrim vs. the World         2010   
7                        The Nice Guys         2016   

                                             Synopsis  \
8   A young Han Solo (Alden Ehrenreich) joins a gr...   
28  En esta comedia española, un policía corrupto ...   
12  Cette comédie romantique raconte l'histoire d'...   
1   Batman (Christian Bale) teams up with District...   
0   Andy Dufresne (Tim Robbins), a successful bank...   
16  Deux employés de bureau incompétents se retrou...   
26  En esta comedia española, un grupo de personas...   
25  Esta serie de televisión española sigue las av...   
6   Scott Pilgrim (Michael Cera) must defeat his n...   
7   In 1970s Los Angeles, a private eye (Ryan Gosl...   

                                               Review Original Language  
8   "Dull and pointless, with none of the magic of...                en  
28  "Torrente es una película vulgar y ofensiva qu...                sp  
12  "Amélie est un film absolument charmant qui vo...                fr  
1   "The Dark Knight is a thrilling and intense su...                en  
0   "The Shawshank Redemption is an inspiring tale...                en  
16  "Je ne peux pas croire que j'ai perdu du temps...                fr  
26  "Toc Toc es una película aburrida y poco origi...                sp  
25  "Águila Roja es una serie aburrida y poco inte...                sp  
6   "It was difficult to sit through the whole thi...                en  
7   "The Nice Guys tries too hard to be funny, and...                en

### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [4]:
# load translation models and tokenizers

fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt")

    # generate the translation using the model
    outputs  = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

c:\Users\jehad\OneDrive\Documents\GitHub\deep-learning-v2-pytorch\DLENV\lib\site-packages\transformers\models\marian\tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [5]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_reviews = df[df["Original Language"] == "fr"]["Review"]

fr_reviews_en = fr_reviews.apply(lambda x: translate(x, fr_en_model, fr_en_tokenizer))

# filter synopsis in French and translate to English
fr_synopsis = df[df["Original Language"] == "fr"]["Synopsis"]
fr_synopsis_en = fr_synopsis.apply(lambda x: translate(x, fr_en_model, fr_en_tokenizer))

# filter reviews in Spanish and translate to English
es_reviews = df[df["Original Language"] == "sp"]["Review"]
es_reviews_en = es_reviews.apply(lambda x: translate(x, es_en_model, es_en_tokenizer))

# filter synopsis in Spanish and translate to English
es_synopsis = df[df["Original Language"] == "sp"]["Synopsis"]
es_synopsis_en = es_synopsis.apply(lambda x: translate(x, es_en_model, es_en_tokenizer))

# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data
df.loc[df["Original Language"] == "fr", "Review"] = fr_reviews_en
df.loc[df["Original Language"] == "fr", "Synopsis"] = fr_synopsis_en
df.loc[df["Original Language"] == "sp", "Review"] = es_reviews_en
df.loc[df["Original Language"] == "sp", "Synopsis"] = es_synopsis_en

In [6]:
df

Title         Year  \
0             The Shawshank Redemption          1994   
1                      The Dark Knight          2008   
2                          Forrest Gump         1994   
3                        The Godfather          1972   
4                             Inception         2010   
5                     Blade Runner 2049         2017   
6           Scott Pilgrim vs. the World         2010   
7                         The Nice Guys         2016   
8               Solo: A Star Wars Story         2018   
9                            The Island         2005   
10                           La La Land         2016   
11                         Intouchables         2011   
12                               Amélie         2001   
13                        Les Choristes         2004   
14  Le Fabuleux Destin d'Amélie Poulain         2001   
15                     Le Dîner de Cons         1998   
16       La Tour Montparnasse Infernale         2001   
17          Astérix aux Jeux Olympiques         2008   
18            Les Visiteurs en Amérique         2000   
19                         Babylon A.D.         2008   
20                                 Roma         2018   
21                     La Casa de Papel  (2017-2021)   
22                    Y tu mamá también         2001   
23               El Laberinto del Fauno         2006   
24                        Amores perros         2000   
25                         Águila Roja   (2009-2016)   
26                              Toc Toc         2017   
27                               El Bar         2017   
28   Torrente: El brazo tonto de la ley         1998   
29                         El Incidente         2014   

                                             Synopsis  \
0   Andy Dufresne (Tim Robbins), a successful bank...   
1   Batman (Christian Bale) teams up with District...   
2   Forrest Gump (Tom Hanks) is a simple man with ...   
3   Don Vito Corleone (Marlon Brando) is the head ...   
4   Dom Cobb (Leonardo DiCaprio) is a skilled thie...   
5   Officer K (Ryan Gosling), a new blade runner f...   
6   Scott Pilgrim (Michael Cera) must defeat his n...   
7   In 1970s Los Angeles, a private eye (Ryan Gosl...   
8   A young Han Solo (Alden Ehrenreich) joins a gr...   
9   In a future where people are cloned for organ ...   
10  This musical tells the story of a budding actr...   
11  This film tells the story of the unlikely frie...   
12  This romantic comedy tells the story of Amélie...   
13  This film tells the story of a music teacher w...   
14  This romantic comedy tells the story of Amélie...   
15  The film follows the story of a group of rich ...   
16  Two incompetent office workers find themselves...   
17  In this film adaptation of the popular comic s...   
18  In this continuation of the French comedy The ...   
19  In the distant future, a mercenary has to esco...   
20  Cleo (Yalitza Aparicio) is a young domestic wo...   
21  This Spanish television series follows a group...   
22  Two teenage friends (Gael García Bernal and Di...   
23  During the Spanish postwar period, Ofelia (Iva...   
24  Three stories intertwine in this Mexican film:...   
25  This Spanish television series follows the adv...   
26  In this Spanish comedy, a group of people with...   
27  A group of people are trapped in a bar after M...   
28  In this Spanish comedy, a corrupt cop (played ...   
29  In this Mexican horror film, a group of people...   

                                               Review Original Language  
0   "The Shawshank Redemption is an inspiring tale...                en  
1   "The Dark Knight is a thrilling and intense su...                en  
2   "Forrest Gump is a heartwarming and inspiratio...                en  
3   "The Godfather is a classic movie that stands ...                en  
4   "Inception is a mind-bending and visually stun...                en  
5   "Boring and too long. Nothing like the origina...                en  
6   "It was

### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [7]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_classifier = pipeline("sentiment-analysis", model=model_name)

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    # perform sentiment analysis and return the result
    result = classifier(text)
    return result[0]["label"]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

c:\Users\jehad\OneDrive\Documents\GitHub\deep-learning-v2-pytorch\DLENV\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jehad\.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [8]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
df["Review Sentiment"] = df["Review"].apply(lambda x: analyze_sentiment(x, sentiment_classifier))

In [9]:
df.sample(10)

Title         Year  \
5                Blade Runner 2049         2017   
16  La Tour Montparnasse Infernale         2001   
26                         Toc Toc         2017   
15                Le Dîner de Cons         1998   
12                          Amélie         2001   
6      Scott Pilgrim vs. the World         2010   
4                        Inception         2010   
23          El Laberinto del Fauno         2006   
21                La Casa de Papel  (2017-2021)   
1                 The Dark Knight          2008   

                                             Synopsis  \
5   Officer K (Ryan Gosling), a new blade runner f...   
16  Two incompetent office workers find themselves...   
26  In this Spanish comedy, a group of people with...   
15  The film follows the story of a group of rich ...   
12  This romantic comedy tells the story of Amélie...   
6   Scott Pilgrim (Michael Cera) must defeat his n...   
4   Dom Cobb (Leonardo DiCaprio) is a skilled thie...   
23  During the Spanish postwar period, Ofelia (Iva...   
21  This Spanish television series follows a group...   
1   Batman (Christian Bale) teams up with District...   

                                               Review Original Language  \
5   "Boring and too long. Nothing like the origina...                en   
16  "I can't believe I've wasted time watching thi...                fr   
26  "Toc Toc is a boring and unoriginal film that ...                sp   
15  "I didn't like this movie at all. The concept ...                fr   
12  "Amélie is an absolutely charming film that wi...                fr   
6   "It was difficult to sit through the whole thi...                en   
4   "Inception is a mind-bending and visually stun...                en   
23  "The Labyrinth of Fauno is a fascinating and e...                sp   
21  "The Paper House is an exciting and addictive ...                sp   
1   "The Dark Knight is a thrilling and intense su...                en   

   Review Sentiment  
5          NEGATIVE  
16         NEGATIVE  
26         NEGATIVE  
15         NEGATIVE  
12         POSITIVE  
6          NEGATIVE  
4          POSITIVE  
23         POSITIVE  
21         POSITIVE  
1          POSITIVE

In [10]:
# export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)